## Natural Language Inference

Zero-shot classification can also be done using [Natural Language Inference (NLI)](https://nlpprogress.com/english/natural_language_inference.html), which refers to the task of determining, given a "premise" and a "hypothesis", whether the hypothesis is:

- True (**entailment**)
- False (**contradiction**)
- Undetermined (**neutral**)

Example:

| Premise | Hypothesis | Label |
| --- | --- | --- |
| A soccer game with multiple males playing. | Some men are playing a sport. | entailment |
| A man inspects the uniform of a figure in some East Asian country. | The man is sleeping. | contradiction |
| An older and younger man smiling. | Two men are smiling and laughing at the cats playing on the floor. | neutral |

NLI (Natural Language Inference) can pull off zero-shot classification by turning the task into a true/false question. Here's how:
1. take the text you want to classify (e.g., a movie review) and call it the "premise."
2. craft a "hypothesis" like, “This is a positive review.”
3. The model checks if this hypothesis follows from the premise (entailment = true) or contradicts it (false).
    - If it "entails," label it positive;
    - if it "contradicts," it’s negative.

You don't even need specific training for this.

Zero, single and few-shot classification seem to be an emergent feature of large language models. This feature seems to come about around model sizes of +100M parameters. The effectiveness of a model at a zero, single or few-shot task seems to scale with model size, meaning that larger models (models with more trainable parameters or layers) generally do better at this task.

- Approaches used for NLI include earlier symbolic and statistical approaches to more recent deep learning approaches.
- Benchmark datasets used for NLI include [SNLI](https://paperswithcode.com/dataset/snli), [MultiNLI](https://paperswithcode.com/dataset/multinli), [SciTail](https://paperswithcode.com/dataset/scitail), among others.
- You can get hands-on practice on the SNLI task by following this [d2l.ai chapter](https://d2l.ai/chapter_natural-language-processing-applications/natural-language-inference-and-dataset.html).

Let's grab a [NLI model from HuggingFace](https://huggingface.co/models?pipeline_tag=zero-shot-classification) and demonstrate how to use it for zero-shot classification:

In [3]:
%pip install -qU datasets transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 104.9 MB/s eta 0:00:00


In [4]:
from transformers import pipeline

# Pre-trained MNLI model
pipe = pipeline(
    model="facebook/bart-large-mnli",
    device='cuda',
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda


In [5]:
predictions = pipe("I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)
predictions

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5227577686309814,
  0.45814040303230286,
  0.014264780096709728,
  0.0026850062422454357,
  0.0021520727314054966]}

Let's make it multi-label classification via `multi_labels=True`:

In [6]:
predictions = pipe("I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
    multi_label=True
)
predictions

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.9987171292304993,
  0.9945850968360901,
  0.18989649415016174,
  0.0007674150983802974,
  0.0003826087631750852]}

Let's try running this on the `rotten_tomatoes` dataset (movie reviews):

In [7]:
import pandas as pd
from datasets import load_dataset

tomatoes = load_dataset("rotten_tomatoes")

# Pandas for easier control
# tomatoes_train_df = pd.DataFrame(tomatoes["train"])
tomatoes_eval_df = pd.DataFrame(tomatoes["test"])

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [15]:
tomatoes_eval_df

,text,label
0,lovingly photographed in the manner of a golde...,1
1,consistently clever and suspenseful .,1
2,"it's like a "" big chill "" reunion of the baade...",1
3,the story gives ample opportunity for large-sc...,1
4,"red dragon "" never cuts corners .",1
...,...,...
1061,a terrible movie that some people will neverth...,0
1062,there are many definitions of 'time waster' bu...,0
1063,"as it stands , crocodile hunter has the hurrie...",0
1064,the thing looks like a made-for-home-video qui...,0


It takes 44s to classify all 1,066 examples on a run on T4 GPU:

In [9]:
# Candidate labels
candidate_labels = [
    "very negative movie review",
    "very positive movie review",
]
candidate_labels_dict = {k: v for k, v in enumerate(candidate_labels)}

# Create predictions
predictions = pipe(tomatoes_eval_df.text.tolist(), candidate_labels)

In [17]:
predictions

[{'sequence': 'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .',
  'labels': ['very positive movie review', 'very negative movie review'],
  'scores': [0.9860209822654724, 0.013979016803205013]},
 {'sequence': 'consistently clever and suspenseful .',
  'labels': ['very positive movie review', 'very negative movie review'],
  'scores': [0.9858868718147278, 0.014113093726336956]},
 {'sequence': 'it\'s like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .',
  'labels': ['very positive movie review', 'very negative movie review'],
  'scores': [0.5961082577705383, 0.40389180183410645]},
 {'sequence': 'the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .',
  'labels': ['very positive movie review', 'very negative movie review'],
  'scores': [0.9758872985

## Exercise

Identify the user intent using NLI:

- `"Hello, I want to get me a Laptop how much does it cost?"`
    - `"BUY Laptop"`
- `"I am very frustrated with your service, and I wanna cancel right now!"`
    - `"CANCEL Subscription"`
- `"Here I bought this Keyboard, but it is not working. I want to get my money back!"`
    - `"REFUND Transaction"`

In [11]:
# YOUR CODE HERE
# Testing your NLI example
result = pipe(
    "Here I bought this Keyboard, but it is not working. I want to get my money back!",
    candidate_labels=["BUY Laptop", "CANCEL Subscription", "REFUND Transaction"]
)

result

{'sequence': 'Here I bought this Keyboard, but it is not working. I want to get my money back!',
 'labels': ['REFUND Transaction', 'CANCEL Subscription', 'BUY Laptop'],
 'scores': [0.8885068297386169, 0.06699260324239731, 0.04450051859021187]}

In [12]:
result = pipe(
    "I am very frustrated with your service, and I wanna cancel right now!",
    candidate_labels=["BUY Laptop", "CANCEL Subscription", "REFUND Transaction"]
)

result

{'sequence': 'I am very frustrated with your service, and I wanna cancel right now!',
 'labels': ['CANCEL Subscription', 'REFUND Transaction', 'BUY Laptop'],
 'scores': [0.8480133414268494, 0.13112331926822662, 0.02086329460144043]}

In [13]:

result = pipe(
    "Hello, I want to get me a Laptop how much does it cost?",
    candidate_labels=["BUY Laptop", "CANCEL Subscription", "REFUND Transaction"]
)

result


{'sequence': 'Hello, I want to get me a Laptop how much does it cost?',
 'labels': ['BUY Laptop', 'REFUND Transaction', 'CANCEL Subscription'],
 'scores': [0.8006560802459717, 0.1742575466632843, 0.025086354464292526]}